In [167]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

In [168]:
# read Ospiti.csv
df = pd.read_csv('input/Ospiti.csv')
df["Data"] = pd.to_datetime(df["Data"]).dt.date
df.set_index("Data",inplace=True)
df = df.iloc[365:]
df

,Ospiti
Data,
2022-06-02,557.0
2022-06-03,357.0
2022-06-04,525.0
2022-06-05,459.0
2022-06-06,282.0
...,...
2022-11-26,732.0
2022-11-27,495.0
2022-11-28,226.0


In [169]:
# compare the results of the three models
autoarima = pd.read_csv('output/predictions_autoarima_avg.csv')
autoarima = autoarima.iloc[:-6]
autoarima["Data"] = pd.to_datetime(autoarima["Data"]).dt.date
autoarima.set_index("Data",inplace=True)
autoarima

,AutoArima Forecasts
Data,
2022-06-02,397.950219
2022-06-03,459.580753
2022-06-04,634.782739
2022-06-05,523.540613
2022-06-06,217.089558
...,...
2022-11-26,709.156796
2022-11-27,526.326570
2022-11-28,225.334520


In [170]:
dense = pd.read_csv('output/predictions_dense_avg.csv')
dense.rename(columns={'Unnamed: 0': 'Data'}, inplace=True)
dense = dense.iloc[:-6]
dense["Data"] = pd.to_datetime(dense["Data"]).dt.date
dense.set_index("Data",inplace=True)
dense

,Dense NN forecasts
Data,
2022-06-02,372.70490
2022-06-03,414.35425
2022-06-04,520.87900
2022-06-05,499.16925
2022-06-06,355.03146
...,...
2022-11-26,576.60675
2022-11-27,403.56393
2022-11-28,266.08344


In [171]:
dense_ex = pd.read_csv('output/predictions_dense_ex_avg.csv')
dense_ex.rename(columns={'Unnamed: 0': 'Data',"Dense NN forecasts":"Dense NN forecasts ex"}, inplace=True)
dense_ex = dense_ex.iloc[:-6]
dense_ex["Data"] = pd.to_datetime(dense_ex["Data"]).dt.date
dense_ex.set_index("Data",inplace=True)
dense_ex

,Dense NN forecasts ex
Data,
2022-06-02,300.75810
2022-06-03,330.54040
2022-06-04,567.00354
2022-06-05,554.03986
2022-06-06,324.33563
...,...
2022-11-26,582.70044
2022-11-27,479.06128
2022-11-28,264.70837


In [172]:
# concatenate the four dataframes
df = pd.concat([df, autoarima, dense, dense_ex], axis=1)
df

,Ospiti,AutoArima Forecasts,Dense NN forecasts,Dense NN forecasts ex
Data,,,,
2022-06-02,557.0,397.950219,372.70490,300.75810
2022-06-03,357.0,459.580753,414.35425,330.54040
2022-06-04,525.0,634.782739,520.87900,567.00354
2022-06-05,459.0,523.540613,499.16925,554.03986
2022-06-06,282.0,217.089558,355.03146,324.33563
...,...,...,...,...
2022-11-26,732.0,709.156796,576.60675,582.70044
2022-11-27,495.0,526.326570,403.56393,479.06128
2022-11-28,226.0,225.334520,266.08344,264.70837


In [173]:
#df["Dense NN forecasts"] = (df["Dense NN forecasts"] + df["Ospiti"]) /2
#df["Dense NN forecasts ex"] = (df["Dense NN forecasts ex"] + df["Ospiti"]) /2

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182 entries, 2022-06-02 to 2022-11-30
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ospiti                 182 non-null    float64
 1   AutoArima Forecasts    182 non-null    float64
 2   Dense NN forecasts     182 non-null    float64
 3   Dense NN forecasts ex  182 non-null    float64
dtypes: float64(4)
memory usage: 7.1+ KB


In [175]:
# plot the results with plotly express
fig = px.line(df, title='Ospiti')
fig.show()


In [176]:
# calculate MAPE throught time for each model
mape = pd.DataFrame()
mape["Autoarima"] = ( abs(df["Ospiti"] - df["AutoArima Forecasts"]) / df["Ospiti"] )*100
mape["Dense NN"] = ( abs(df["Ospiti"] - df["Dense NN forecasts"]) / df["Ospiti"] ) *100
mape["Dense NN ex"] = ( abs(df["Ospiti"] - df["Dense NN forecasts ex"]) ) / df["Ospiti"] *100
mape

,Autoarima,Dense NN,Dense NN ex
Data,,,
2022-06-02,28.554718,33.087092,46.003932
2022-06-03,28.734104,16.065616,7.411653
2022-06-04,20.910998,0.784952,8.000674
2022-06-05,14.061136,8.751471,20.705852
2022-06-06,23.017887,25.897681,15.012635
...,...,...,...
2022-11-26,3.120656,21.228586,20.396115
2022-11-27,6.328600,18.471933,3.219943
2022-11-28,0.294460,17.736035,17.127597


In [177]:
mape.describe()

,Autoarima,Dense NN,Dense NN ex
count,182.000000,182.000000,182.000000
mean,14.816926,14.990358,16.966318
std,13.347048,11.797850,14.962965
min,0.011597,0.004061,0.235686
25%,6.242201,6.250526,7.335557
50%,11.352623,12.462221,13.065798
75%,19.039218,20.025492,22.774060
max,97.768777,67.665883,95.302856


In [178]:
mape.median()

Autoarima      11.352623
Dense NN       12.462221
Dense NN ex    13.065798
dtype: float64

In [179]:
# plot MAPE throught time for each model
fig = px.line(mape, title='MAPE')
fig.show()

In [180]:
# calculate moving average of MAE
mape_ma = mape.copy()
mape_ma["Autoarima"] = mape["Autoarima"].rolling(30).mean()
mape_ma["Dense NN"] = mape["Dense NN"].rolling(30).mean()
mape_ma["Dense NN ex"] = mape["Dense NN ex"].rolling(30).mean()
mape_ma

,Autoarima,Dense NN,Dense NN ex
Data,,,
2022-06-02,NaN,NaN,NaN
2022-06-03,NaN,NaN,NaN
2022-06-04,NaN,NaN,NaN
2022-06-05,NaN,NaN,NaN
2022-06-06,NaN,NaN,NaN
...,...,...,...
2022-11-26,19.260704,19.676156,18.590868
2022-11-27,19.212889,20.026923,18.584089
2022-11-28,18.249270,20.372009,18.875109


In [181]:
mape_ma.describe()

,Autoarima,Dense NN,Dense NN ex
count,153.000000,153.000000,153.000000
mean,14.653676,14.943708,17.411925
std,3.826559,3.241943,4.068280
min,7.467802,9.145347,11.427180
25%,11.935842,12.595875,13.986863
50%,14.654892,15.032880,17.160457
75%,17.552138,17.557418,20.747074
max,22.000798,20.488654,26.028560


In [182]:
mape_ma.median()

Autoarima      14.654892
Dense NN       15.032880
Dense NN ex    17.160457
dtype: float64

In [183]:
# calculate moving average of MAE
mape_ma_7 = mape.copy()
mape_ma_7["Autoarima"] = mape["Autoarima"].rolling(7).mean()
mape_ma_7["Dense NN"] = mape["Dense NN"].rolling(7).mean()
mape_ma_7["Dense NN ex"] = mape["Dense NN ex"].rolling(7).mean()
mape_ma_7

,Autoarima,Dense NN,Dense NN ex
Data,,,
2022-06-02,NaN,NaN,NaN
2022-06-03,NaN,NaN,NaN
2022-06-04,NaN,NaN,NaN
2022-06-05,NaN,NaN,NaN
2022-06-06,NaN,NaN,NaN
...,...,...,...
2022-11-26,9.826682,11.268678,11.156871
2022-11-27,9.087044,11.470804,7.681319
2022-11-28,5.520274,10.082721,8.732387


In [184]:
mape_ma_7.describe()

,Autoarima,Dense NN,Dense NN ex
count,176.000000,176.000000,176.000000
mean,14.728851,14.926054,16.941798
std,6.958385,5.328847,6.638279
min,3.793143,4.408128,7.314365
25%,9.665652,11.306853,11.400559
50%,12.914626,13.760259,15.298008
75%,19.451564,18.119747,22.019818
max,35.644241,31.816637,35.227673


In [185]:
mape.median()

Autoarima      11.352623
Dense NN       12.462221
Dense NN ex    13.065798
dtype: float64

In [186]:
# plot moving average of MAE
fig = px.line(mape_ma, title='MA MAPE')
fig.show()

In [187]:
# last 30 days of MAPE
mape_last_30 = mape.tail(30).copy()
mape_last_30.describe()

,Autoarima,Dense NN,Dense NN ex
count,30.000000,30.000000,30.000000
mean,17.376409,19.087449,18.161842
std,19.364382,15.134826,18.845937
min,0.011597,1.333685,0.287453
25%,5.266312,8.538734,4.875534
50%,11.179429,17.201117,10.317325
75%,25.652753,21.345455,23.125096
max,97.768777,67.665883,80.614583


In [188]:
mape_last_30.median()

Autoarima      11.179429
Dense NN       17.201117
Dense NN ex    10.317325
dtype: float64